In [ ]:
%load_ext autoreload
%autoreload 2
from main import init_model, training_loop

tokenizer, model = init_model()

In [ ]:
model.device

In [3]:
import pandas as pd
df = pd.read_json("connections_prompts.jsonl", lines=True)

In [ ]:
df.iloc[0]

In [35]:
from dataset import create_connections_datasets
from main import get_available_device
train_dataset, _ = create_connections_datasets(tokenizer)
device = get_available_device()

In [41]:
train_dataset.get_collate_fn()

<function dataset.get_collate_fn.<locals>._collate_fn(batch: 'list[dict]') -> 'MiniBatch'>

In [42]:
training_loop(model, tokenizer, train_dataset, device)


<function get_collate_fn.<locals>._collate_fn at 0x15d0ce340>


TypeError: TextInputSequence must be str